In [1]:
from precomputed_python import AnnotationReader, morton_code_to_gridpt, compressed_morton_code
import numpy as np
import pandas as pd
import asyncio
import tensorstore as ts
from neuroglancer import viewer_state

In [2]:
# pre_ann=AnnotationReader("gs://neuroglancer-20191211_fafbv14_buhmann2019_li20190805")
# ann_id= 15959574
# ann = pre_ann.get_by_id(ann_id)

pre_ann = AnnotationReader("gs://allen_neuroglancer_ccf/merscope_ccffix_1")
ann_id= 1813224
ann = pre_ann.get_by_id(1813224)


I0402 18:31:02.347001 9912288 google_auth_provider.cc:149] Using credentials at /Users/forrestc/.config/gcloud/application_default_credentials.json
I0402 18:31:02.348156 9912288 google_auth_provider.cc:156] Using OAuth2 AuthProvider


In [3]:
ann

{'regionset_1_cluster_idx': np.int32(64),
 'regionset_2_cluster_idx': np.int32(261),
 'regionset_3_cluster_idx': np.int32(449),
 'neurotransmitter': 'Glut',
 'class': '18 TH Glut',
 'subclass': '154 PF Fzd5 Glut',
 'supertype': '0675 PF Fzd5 Glut_2',
 'cluster': '2722 PF Fzd5 Glut_2',
 'neurotransmitter_color': array([ 43, 147, 223], dtype=uint8),
 'class_color': array([ 13,  71, 161], dtype=uint8),
 'subclass_color': array([ 28,   0, 153], dtype=uint8),
 'supertype_color': array([ 69,  69, 153], dtype=uint8),
 'cluster_color': array([ 15, 102,  16], dtype=uint8),
 'regionset_1_cluster_color': array([218, 149, 255], dtype=uint8),
 'regionset_2_cluster_color': array([255, 126, 209], dtype=uint8),
 'regionset_3_cluster_color': array([230, 175,   0], dtype=uint8),
 'padding46': array([0, 0], dtype=uint8),
 'ccf_ids': array([997,   8, 549, 362], dtype=uint64),
 'cluster_annotation_index': array([4304, 1056,  197,   27,    0], dtype=uint64),
 'point': array([7347.7915, 3566.362 , 6301.7676]

In [4]:
pre_ann.read_annotations_in_chunk('spatial0', chunk_index=[3,1,8])

KvStore.ReadResult(state='value', value=b'f\x00\x00\x00\x00\x00\x00\x00LxTE`\x90\xfcD9\xa8\x01F\xb7\x00\x00\x00b\x01\x00\x00\x1b\x01\x00\x00\x00\x00\r\x00P\x00\x92\x00\x01\x01fff\x85\x88\x81\xcc\\\xc1U=f&\xff\xe8/W&\x82\x01\xcf\xa5{\xb8\x00\x00R|TE\xf5\xbe\xfbD\xf9o\x01F\xb7\x00\x00\x00I\x00\x00\x00\x1b\x01\x00\x00\x00\x00\r\x00P\x00\x92\x00\x01\x01fff\x85\x88\x81\xcc\\\xc1U=f&\xff\xe8/W&\xffF\x00\xa5{\xb8\x00\x00%\x16lE\x1c\xee\xe5DiG\x00F\x0c\x00\x00\x00\x19\x01\x00\x00&\x00\x00\x00\x04\x00\x1a\x00X\x00\xdc\x01s\x03\xff3X\xcc\xff3\xffvM\xc0\xcc=f$\x1f\xa1ui\x9a\xe4\xff\xfd\xbf\xff\x00\x00\xab4`EF)\xf1DkP\x00F\x0c\x00\x00\x00\x19\x01\x00\x00\xcc\x00\x00\x00\x00\x00\x07\x00\n\x00?\x00j\x01fffYJ&=\xcc\xb1\xa8\xaf\xa5U\x17\x99\xa1ui\x9a\xe4\xff\xe2A/\x00\x00\xbd,kE\xaa-\xf8D#g\x00F\x18\x00\x00\x00\xc7\x01\x00\x00\xcb\x01\x00\x00\x01\x00\x0b\x00d\x00\xb8\x00J\x01+\x93\xdf\xfa\x00\x87\xe9S\x0f\xccz\x8f\xaaz\xcc]~fy\x9e\xaf\x8emI\x00\x00\xa1\x16kE\x18\xf1\xf9D\xf8m\x00F\x18\x00\x00\x00\xc7\

In [5]:
pre_ann.get_relationships()

['rel_ccf_ids', 'rel_cluster_annotation_index']

In [6]:
pre_ann.get_property_names()

['regionset_1_cluster_idx',
 'regionset_2_cluster_idx',
 'regionset_3_cluster_idx',
 'neurotransmitter',
 'class',
 'subclass',
 'supertype',
 'cluster',
 'neurotransmitter_color',
 'class_color',
 'subclass_color',
 'supertype_color',
 'cluster_color',
 'regionset_1_cluster_color',
 'regionset_2_cluster_color',
 'regionset_3_cluster_color']

In [7]:
pre_ann.properties

[AnnotationPropertySpec({"description": "regionset_1_cluster_idx", "id": "regionset_1_cluster_idx", "type": "int32"}),
 AnnotationPropertySpec({"description": "regionset_2_cluster_idx", "id": "regionset_2_cluster_idx", "type": "int32"}),
 AnnotationPropertySpec({"description": "regionset_3_cluster_idx", "id": "regionset_3_cluster_idx", "type": "int32"}),
 AnnotationPropertySpec({"description": "neurotransmitter", "enum_labels": ["None", "Glut", "Chol", "GABA-Glyc", "GABA", "Dopa", "Glut-GABA", "Sero", "Nora", "Hist"], "enum_values": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], "id": "neurotransmitter", "type": "uint16"}),
 AnnotationPropertySpec({"description": "class", "enum_labels": ["04 DG-IMN Glut", "23 P Glut", "31 OPC-Oligo", "24 MY Glut", "19 MB Glut", "26 P GABA", "05 OB-IMN GABA", "30 Astro-Epen", "27 MY GABA", "09 CNU-LGE GABA", "28 CB GABA", "01 IT-ET Glut", "12 HY GABA", "33 Vascular", "20 MB GABA", "29 CB Glut", "11 CNU-HYa GABA", "03 OB-CR Glut", "34 Immune", "32 OEC", "13 CNU-HYa Glu

In [8]:
pre_ann.coordinate_space

CoordinateSpace({'x': [np.float64(1e-06), 'm'], 'y': [np.float64(1e-06), 'm'], 'z': [np.float64(1e-06), 'm']})

In [9]:
all_ids = pre_ann.get_all_annotation_ids()
all_ids

array([ 513024,  513025,  513026, ..., 1406973, 1406974, 1406975],
      dtype='>u8')

In [10]:
ann=pre_ann.get_by_id(all_ids[0])
ann

{'regionset_1_cluster_idx': np.int32(189),
 'regionset_2_cluster_idx': np.int32(161),
 'regionset_3_cluster_idx': np.int32(591),
 'neurotransmitter': 'None',
 'class': '30 Astro-Epen',
 'subclass': '319 Astro-TE NN',
 'supertype': '1163 Astro-TE NN_3',
 'cluster': '5225 Astro-TE NN_3',
 'neurotransmitter_color': array([102, 102, 102], dtype=uint8),
 'class_color': array([89, 74, 38], dtype=uint8),
 'subclass_color': array([ 61, 204, 177], dtype=uint8),
 'supertype_color': array([168, 175, 165], dtype=uint8),
 'cluster_color': array([ 85,  23, 153], dtype=uint8),
 'regionset_1_cluster_color': array([156,  49, 144], dtype=uint8),
 'regionset_2_cluster_color': array([161, 198, 255], dtype=uint8),
 'regionset_3_cluster_color': array([119,  54,  24], dtype=uint8),
 'padding46': array([0, 0], dtype=uint8),
 'ccf_ids': array([ 997,    8,  315, 1018,  990], dtype=uint64),
 'cluster_annotation_index': array([6807, 1544,  362,   39,    1], dtype=uint64),
 'point': array([ 8399.962 ,  3337.4412, 

In [13]:
df =pre_ann.get_by_relationship('ccf_ids', ann['ccf_ids'][3])
df

,regionset_1_cluster_idx,regionset_2_cluster_idx,regionset_3_cluster_idx,neurotransmitter,class,subclass,supertype,cluster,neurotransmitter_color,class_color,subclass_color,supertype_color,cluster_color,regionset_1_cluster_color,regionset_2_cluster_color,regionset_3_cluster_color,padding46,point
ID,,,,,,,,,,,,,,,,,,
6455,123,367,347,GABA,05 OB-IMN GABA,039 OB Meis2 Thsd7b Gaba,0147 OB Meis2 Thsd7b Gaba_5,0527 OB Meis2 Thsd7b Gaba_5,"[255, 51, 88]","[27, 67, 50]","[255, 153, 247]","[69, 153, 144]","[61, 204, 148]","[68, 0, 196]","[0, 142, 100]","[96, 52, 93]","[0, 0]","[7921.5556640625, 3557.454345703125, 1150.3876..."
12675,15,188,330,GABA,05 OB-IMN GABA,039 OB Meis2 Thsd7b Gaba,0147 OB Meis2 Thsd7b Gaba_5,0528 OB Meis2 Thsd7b Gaba_5,"[255, 51, 88]","[27, 67, 50]","[255, 153, 247]","[69, 153, 144]","[116, 153, 23]","[191, 3, 184]","[200, 121, 221]","[200, 255, 249]","[0, 0]","[7936.79443359375, 3478.1669921875, 1395.20239..."
24058,12,281,145,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0497 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[153, 69, 152]","[161, 117, 105]","[154, 228, 255]","[198, 232, 61]","[0, 0]","[7856.61572265625, 3418.605224609375, 10787.93..."
44210,183,354,283,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0497 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[153, 69, 152]","[47, 87, 38]","[130, 1, 207]","[165, 123, 184]","[0, 0]","[8407.5732421875, 3344.158203125, 10797.811523..."
45554,183,354,283,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0498 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[102, 0, 8]","[47, 87, 38]","[130, 1, 207]","[165, 123, 184]","[0, 0]","[8116.0283203125, 3270.614501953125, 616.33441..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3438305,183,354,229,None,33 Vascular,333 Endo NN,1193 Endo NN_1,5310 Endo NN_1,"[102, 102, 102]","[133, 136, 129]","[153, 69, 103]","[0, 153, 42]","[255, 180, 115]","[47, 87, 38]","[130, 1, 207]","[230, 214, 124]","[0, 0]","[8417.07421875, 3248.67333984375, 10791.232421..."
3438585,37,367,90,None,33 Vascular,333 Endo NN,1193 Endo NN_1,5309 Endo NN_1,"[102, 102, 102]","[133, 136, 129]","[153, 69, 103]","[0, 153, 42]","[92, 153, 148]","[56, 0, 0]","[0, 142, 100]","[0, 195, 107]","[0, 0]","[7450.7353515625, 3661.760986328125, 10136.818..."
3438748,226,367,347,None,33 Vascular,333 Endo NN,1193 Endo NN_1,5310 Endo NN_1,"[102, 102, 102]","[133, 136, 129]","[153, 69, 103]","[0, 153, 42]","[255, 180, 115]","[137, 100, 123]","[0, 142, 100]","[96, 52, 93]","[0, 0]","[8707.205078125, 2935.191162109375, 1093.52600..."


In [14]:
df =pre_ann.get_by_relationship('ccf_ids', ann['ccf_ids'][3], get_relationships=True)
df

,regionset_1_cluster_idx,regionset_2_cluster_idx,regionset_3_cluster_idx,neurotransmitter,class,subclass,supertype,cluster,neurotransmitter_color,class_color,subclass_color,supertype_color,cluster_color,regionset_1_cluster_color,regionset_2_cluster_color,regionset_3_cluster_color,padding46,ccf_ids,cluster_annotation_index,point
ID,,,,,,,,,,,,,,,,,,,,
6455,123,367,347,GABA,05 OB-IMN GABA,039 OB Meis2 Thsd7b Gaba,0147 OB Meis2 Thsd7b Gaba_5,0527 OB Meis2 Thsd7b Gaba_5,"[255, 51, 88]","[27, 67, 50]","[255, 153, 247]","[69, 153, 144]","[61, 204, 148]","[68, 0, 196]","[0, 142, 100]","[96, 52, 93]","[0, 0]","[997, 8, 315, 1018, 1023]","[2109, 528, 82, 14, 2]","[7921.5557, 3557.4543, 1150.3877]"
12675,15,188,330,GABA,05 OB-IMN GABA,039 OB Meis2 Thsd7b Gaba,0147 OB Meis2 Thsd7b Gaba_5,0528 OB Meis2 Thsd7b Gaba_5,"[255, 51, 88]","[27, 67, 50]","[255, 153, 247]","[69, 153, 144]","[116, 153, 23]","[191, 3, 184]","[200, 121, 221]","[200, 255, 249]","[0, 0]","[997, 8, 315, 1018, 520]","[2110, 528, 82, 14, 2]","[7936.7944, 3478.167, 1395.2024]"
24058,12,281,145,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0497 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[153, 69, 152]","[161, 117, 105]","[154, 228, 255]","[198, 232, 61]","[0, 0]","[997, 8, 315, 1018, 959]","[2079, 516, 79, 12, 0]","[7856.6157, 3418.6052, 10787.938]"
44210,183,354,283,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0497 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[153, 69, 152]","[47, 87, 38]","[130, 1, 207]","[165, 123, 184]","[0, 0]","[997, 8, 315, 1018, 959]","[2079, 516, 79, 12, 0]","[8407.573, 3344.1582, 10797.812]"
45554,183,354,283,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0498 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[102, 0, 8]","[47, 87, 38]","[130, 1, 207]","[165, 123, 184]","[0, 0]","[997, 8, 315, 1018, 959]","[2080, 516, 79, 12, 0]","[8116.0283, 3270.6145, 616.3344]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3438305,183,354,229,None,33 Vascular,333 Endo NN,1193 Endo NN_1,5310 Endo NN_1,"[102, 102, 102]","[133, 136, 129]","[153, 69, 103]","[0, 153, 42]","[255, 180, 115]","[47, 87, 38]","[130, 1, 207]","[230, 214, 124]","[0, 0]","[997, 8, 315, 1018, 959]","[6892, 1574, 376, 42, 1]","[8417.074, 3248.6733, 10791.232]"
3438585,37,367,90,None,33 Vascular,333 Endo NN,1193 Endo NN_1,5309 Endo NN_1,"[102, 102, 102]","[133, 136, 129]","[153, 69, 103]","[0, 153, 42]","[92, 153, 148]","[56, 0, 0]","[0, 142, 100]","[0, 195, 107]","[0, 0]","[997, 8, 315, 1018, 1023]","[6891, 1574, 376, 42, 1]","[7450.7354, 3661.761, 10136.818]"
3438748,226,367,347,None,33 Vascular,333 Endo NN,1193 Endo NN_1,5310 Endo NN_1,"[102, 102, 102]","[133, 136, 129]","[153, 69, 103]","[0, 153, 42]","[255, 180, 115]","[137, 100, 123]","[0, 142, 100]","[96, 52, 93]","[0, 0]","[997, 8, 315, 1018, 990]","[6892, 1574, 376, 42, 1]","[8707.205, 2935.1912, 1093.526]"


In [15]:
df = pre_ann.get_all_annotations()
df.head()

ValueError: Number of annotations (3439816) exceeds the maximum allowed (1000000).

In [16]:
some_ids = df.index[0:10]
pre_ann.get_by_ids(some_ids)

,regionset_1_cluster_idx,regionset_2_cluster_idx,regionset_3_cluster_idx,neurotransmitter,class,subclass,supertype,cluster,neurotransmitter_color,class_color,subclass_color,supertype_color,cluster_color,regionset_1_cluster_color,regionset_2_cluster_color,regionset_3_cluster_color,padding46,ccf_ids,cluster_annotation_index,point
ID,,,,,,,,,,,,,,,,,,,,
6455,123,367,347,GABA,05 OB-IMN GABA,039 OB Meis2 Thsd7b Gaba,0147 OB Meis2 Thsd7b Gaba_5,0527 OB Meis2 Thsd7b Gaba_5,"[255, 51, 88]","[27, 67, 50]","[255, 153, 247]","[69, 153, 144]","[61, 204, 148]","[68, 0, 196]","[0, 142, 100]","[96, 52, 93]","[0, 0]","[997, 8, 315, 1018, 1023]","[2109, 528, 82, 14, 2]","[7921.5557, 3557.4543, 1150.3877]"
12675,15,188,330,GABA,05 OB-IMN GABA,039 OB Meis2 Thsd7b Gaba,0147 OB Meis2 Thsd7b Gaba_5,0528 OB Meis2 Thsd7b Gaba_5,"[255, 51, 88]","[27, 67, 50]","[255, 153, 247]","[69, 153, 144]","[116, 153, 23]","[191, 3, 184]","[200, 121, 221]","[200, 255, 249]","[0, 0]","[997, 8, 315, 1018, 520]","[2110, 528, 82, 14, 2]","[7936.7944, 3478.167, 1395.2024]"
24058,12,281,145,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0497 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[153, 69, 152]","[161, 117, 105]","[154, 228, 255]","[198, 232, 61]","[0, 0]","[997, 8, 315, 1018, 959]","[2079, 516, 79, 12, 0]","[7856.6157, 3418.6052, 10787.938]"
44210,183,354,283,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0497 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[153, 69, 152]","[47, 87, 38]","[130, 1, 207]","[165, 123, 184]","[0, 0]","[997, 8, 315, 1018, 959]","[2079, 516, 79, 12, 0]","[8407.573, 3344.1582, 10797.812]"
45554,183,354,283,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0498 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[102, 0, 8]","[47, 87, 38]","[130, 1, 207]","[165, 123, 184]","[0, 0]","[997, 8, 315, 1018, 959]","[2080, 516, 79, 12, 0]","[8116.0283, 3270.6145, 616.3344]"
45644,122,354,28,Glut,03 OB-CR Glut,036 HPF CR Glut,0135 HPF CR Glut_1,0500 HPF CR Glut_1,"[43, 147, 223]","[208, 0, 0]","[145, 153, 0]","[102, 46, 89]","[0, 82, 153]","[205, 133, 188]","[130, 1, 207]","[89, 0, 163]","[0, 0]","[997, 8, 315, 1018, 959]","[2082, 516, 79, 12, 0]","[8683.667, 3048.1323, 682.43524]"
79682,123,188,330,GABA,05 OB-IMN GABA,041 OB-in Frmd7 Gaba,0150 OB-in Frmd7 Gaba_1,0541 OB-in Frmd7 Gaba_1,"[255, 51, 88]","[27, 67, 50]","[255, 0, 115]","[166, 255, 153]","[204, 61, 140]","[68, 0, 196]","[200, 121, 221]","[200, 255, 249]","[0, 0]","[997, 8, 315, 1018, 520]","[2123, 531, 84, 14, 2]","[7936.1504, 3479.6873, 1378.5271]"
252546,199,226,116,GABA,05 OB-IMN GABA,043 OB-mi Frmd7 Gaba,0161 OB-mi Frmd7 Gaba_1,0579 OB-mi Frmd7 Gaba_1,"[255, 51, 88]","[27, 67, 50]","[255, 153, 167]","[113, 92, 204]","[61, 204, 200]","[121, 158, 175]","[137, 100, 123]","[209, 182, 91]","[0, 0]","[997, 8, 315, 1018, 959]","[2161, 542, 86, 14, 2]","[7669.5024, 3470.491, 10760.37]"
252867,199,459,116,GABA,06 CTX-CGE GABA,046 Vip Gaba,0178 Vip Gaba_6,0645 Vip Gaba_6,"[255, 51, 88]","[204, 255, 51]","[102, 61, 71]","[122, 150, 204]","[153, 104, 46]","[121, 158, 175]","[142, 109, 73]","[209, 182, 91]","[0, 0]","[997, 8, 315, 1018, 959]","[2227, 559, 89, 15, 2]","[7651.3354, 3645.6636, 10747.347]"


In [4]:
box_size = [500, 500, 500]
point = ann.get('point', ann.get('point_a',None))
lower_bound = point-box_size
upper_bound = point+box_size

df=pre_ann.get_annotations_in_bounds(lower_bound=lower_bound,
                                     upper_bound=upper_bound)
df

,regionset_1_cluster_idx,regionset_2_cluster_idx,regionset_3_cluster_idx,neurotransmitter,class,subclass,supertype,cluster,neurotransmitter_color,class_color,subclass_color,supertype_color,cluster_color,regionset_1_cluster_color,regionset_2_cluster_color,regionset_3_cluster_color,padding46,point
ID,,,,,,,,,,,,,,,,,,
1171140,118,418,205,None,30 Astro-Epen,318 Astro-NT NN,1159 Astro-NT NN_1,5210 Astro-NT NN_1,"[102, 102, 102]","[89, 74, 38]","[173, 92, 204]","[53, 0, 153]","[153, 46, 71]","[153, 84, 68]","[144, 161, 233]","[184, 73, 107]","[0, 0]","[6848.84130859375, 3068.554443359375, 6084.521..."
1334619,118,418,205,None,31 OPC-Oligo,327 Oligo NN,1184 MOL NN_4,5286 MOL NN_4,"[102, 102, 102]","[3, 4, 94]","[153, 255, 188]","[153, 202, 255]","[159, 255, 153]","[153, 84, 68]","[144, 161, 233]","[184, 73, 107]","[0, 0]","[6849.42578125, 3069.22802734375, 6106.7490234..."
1321622,118,418,205,None,31 OPC-Oligo,327 Oligo NN,1184 MOL NN_4,5286 MOL NN_4,"[102, 102, 102]","[3, 4, 94]","[153, 255, 188]","[153, 202, 255]","[159, 255, 153]","[153, 84, 68]","[144, 161, 233]","[184, 73, 107]","[0, 0]","[6848.3876953125, 3067.8955078125, 6066.810546..."
1678645,118,418,205,Glut,17 MH-LH Glut,146 LH Pou4f1 Sox1 Glut,0639 LH Pou4f1 Sox1 Glut_3,2606 LH Pou4f1 Sox1 Glut_3,"[43, 147, 223]","[250, 163, 7]","[204, 122, 157]","[0, 62, 153]","[204, 188, 92]","[153, 84, 68]","[144, 161, 233]","[184, 73, 107]","[0, 0]","[6848.14208984375, 3071.078369140625, 6069.451..."
1678666,118,418,205,Glut,17 MH-LH Glut,146 LH Pou4f1 Sox1 Glut,0639 LH Pou4f1 Sox1 Glut_3,2606 LH Pou4f1 Sox1 Glut_3,"[43, 147, 223]","[250, 163, 7]","[204, 122, 157]","[0, 62, 153]","[204, 188, 92]","[153, 84, 68]","[144, 161, 233]","[184, 73, 107]","[0, 0]","[6848.60498046875, 3068.387939453125, 6075.902..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756907,64,264,288,None,31 OPC-Oligo,326 OPC NN,1179 OPC NN_1,5269 OPC NN_1,"[102, 102, 102]","[3, 4, 94]","[255, 38, 203]","[204, 0, 73]","[92, 153, 123]","[218, 149, 255]","[87, 59, 0]","[202, 195, 0]","[0, 0]","[7676.0712890625, 3866.748046875, 6189.4887695..."
639988,64,264,288,None,30 Astro-Epen,318 Astro-NT NN,1160 Astro-NT NN_2,5214 Astro-NT NN_2,"[102, 102, 102]","[89, 74, 38]","[173, 92, 204]","[31, 142, 204]","[44, 0, 255]","[218, 149, 255]","[87, 59, 0]","[202, 195, 0]","[0, 0]","[7671.0029296875, 3906.213623046875, 6153.3295..."
3299605,64,264,288,None,33 Vascular,333 Endo NN,1193 Endo NN_1,5311 Endo NN_1,"[102, 102, 102]","[133, 136, 129]","[153, 69, 103]","[0, 153, 42]","[204, 61, 118]","[218, 149, 255]","[87, 59, 0]","[202, 195, 0]","[0, 0]","[7690.06201171875, 3734.711181640625, 6209.449..."
